<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_nlp2.0/blob/week_5_scraping_parsing/parsing_news_from_lenta_ru_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсер новостных текстов с сайтов РБК и Лента.ру

## Импорт библиотек и описание классов

In [21]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [22]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

Ниже описаны классы для извлечения данных.

Используются поисковые движки сайтов, возвращающие по запросу json таблицы с информацией о статьях.

При их использовании есть различные ограничения, например, на количество статей в запросе. Так на сайте РБК выдается максимум 100 статей, в то время как на Ленте.ру можно получить сразу 1000. По моим наблюдениям, запросы на Лента.ру ограничены временем, и соответственно, точное ограничение найти не получится. Более того, на Лента.ру сразу выдается текст статьи в json таблице, с РБК приходится дополнительно парсить текст со страниц.  

### Парсер для Лента.ру

In [23]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict["type"]) != 0
        hasBloc = int(param_dict["bloc"]) != 0

        url = (
            "https://lenta.ru/search/v2/process?"
            + "from={}&".format(param_dict["from"])
            + "size={}&".format(param_dict["size"])
            + "sort={}&".format(param_dict["sort"])
            + "title_only={}&".format(param_dict["title_only"])
            + "domain={}&".format(param_dict["domain"])
            + "modified%2Cformat=yyyy-MM-dd&"
            + "type={}&".format(param_dict["type"]) * hasType
            + "bloc={}&".format(param_dict["bloc"]) * hasBloc
            + "modified%2Cfrom={}&".format(param_dict["dateFrom"])
            + "modified%2Cto={}&".format(param_dict["dateTo"])
            + "query={}".format(param_dict["query"])
        )

        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()["matches"])

        return search_table

    def get_articles(
        self, param_dict, time_step=37, save_every=5, save_excel=True
    ) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy["dateFrom"], "%Y-%m-%d")
        dateTo = datetime.strptime(param_copy["dateTo"], "%Y-%m-%d")
        if dateFrom > dateTo:
            raise ValueError("dateFrom should be less than dateTo")

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy["dateTo"] = (dateFrom + time_step).strftime("%Y-%m-%d")
            if dateFrom + time_step > dateTo:
                param_copy["dateTo"] = dateTo.strftime("%Y-%m-%d")
            print(
                "Parsing articles from "
                + param_copy["dateFrom"]
                + " to "
                + param_copy["dateTo"]
            )
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy["dateFrom"] = dateFrom.strftime("%Y-%m-%d")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print("Checkpoint saved!")
                save_counter = 0

        if save_excel:
            out.to_excel(
                "lenta_{}_{}.xlsx".format(param_dict["dateFrom"], param_dict["dateTo"])
            )
        print("Finish")

        return out

## Пример выгрузки данных

### Лента.ру

* __query__ - поисковой запрос (ключевое слово)

* __offset__ - cмещение поисковой выдачи (от 0 до __size__)

* __size__ - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда

* __sort__ - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1)

* __title_only__ - точная фраза в заголовке (1)

* __domain__ - ?

* __material__ - материалы: Все материалы (0). Новость (1). ["0", "1", "2", "3", "4", ...]

* __block__ - рубрика: Экономика (4). Все рубрики (0). ["0", "1", "2", "3", "4", ...]

* __dateFrom__ - с даты

* __dateTo__ - по дату

_Чтобы не специфировать параметр, оставляем поле пустым_

In [24]:
use_parser = "LentaRu"

query = ""
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0"
dateFrom = "2016-01-01"
dateTo = "2023-10-21"

if use_parser == "LentaRu":
    param_dict = {
        "query": query,
        "from": str(offset),
        "size": str(size),
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "sort": sort,
        "title_only": title_only,
        "type": material,
        "bloc": bloc,
        "domain": domain,
    }

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2016-01-01', 'dateTo': '2023-10-21', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [25]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(
    param_dict=param_dict, time_step=37, save_every=5, save_excel=True
)
print(len(tbl.index))

Checkpoint saved!
Parsing articles from 2023-10-21 to 2023-10-21


<ipython-input-23-e7bc6880237a>:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
75236


In [26]:
# tbl.head()

In [27]:
tbl.shape

(75236, 16)

In [28]:
tbl.to_excel('lenta75000.xlsx',)